## Data Cleaning

#### Import modules

In [6]:
import os
import pandas as pd

#### Define paths and load data

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
PATH = 'raw_data'

In [9]:
OUTPUT_PATH = 'outputs'

### Election results

#### Load data

In [10]:
results = pd.read_csv(os.path.join(PATH, 'HarvardData_countypres_2000-2020.csv'))
results = results[results['party'].isin(['DEMOCRAT', 'REPUBLICAN'])]
grpby_cols = ['year', 'state', 'state_po', 'county_name', 'county_fips', 'office',
              'candidate', 'party', 'totalvotes', 'version']
results = results.groupby(grpby_cols, as_index=False).sum()

In [11]:
results['pctvotes'] = results['candidatevotes'] / results['totalvotes']
results['office'] = 'PRESIDENT'  # for consistency

#### Pivot table

In [12]:
index_cols = ['year', 'state', 'state_po', 'county_name', 
              'county_fips', 'office']

In [13]:
pivoted_results = pd.pivot_table(results, index=index_cols, values=['pctvotes', 'candidatevotes'], columns=['party']).reset_index()
pivoted_results.columns.name = ''
pivoted_results.columns = [(x[0] + '_' + x[1]).lower() if x[1] != '' else x[0] for x in  pivoted_results.columns.values]
a = pivoted_results['candidatevotes_democrat'] > pivoted_results['candidatevotes_republican']
pivoted_results['winner'] = ['democrat' if x else 'republican' for x in a]
pivoted_results = pivoted_results[['year', 'state', 'county_name', 'county_fips', 'candidatevotes_democrat',
                                   'candidatevotes_republican', 'pctvotes_democrat', 'pctvotes_republican', 'winner']]

In [14]:
pivoted_results.head()

,year,state,county_name,county_fips,candidatevotes_democrat,candidatevotes_republican,pctvotes_democrat,pctvotes_republican,winner
0,2000,ALABAMA,AUTAUGA,1001.0,4942.0,11993.0,0.287192,0.696943,republican
1,2000,ALABAMA,BALDWIN,1003.0,13997.0,40872.0,0.247822,0.723654,republican
2,2000,ALABAMA,BARBOUR,1005.0,5188.0,5096.0,0.499086,0.490236,democrat
3,2000,ALABAMA,BIBB,1007.0,2710.0,4273.0,0.381636,0.601746,republican
4,2000,ALABAMA,BLOUNT,1009.0,4977.0,12667.0,0.276915,0.704779,republican


Export election results

In [15]:
for year in pivoted_results['year'].unique():
    
    df_temp = pivoted_results[pivoted_results['year'] == year]
    df_temp.to_csv(os.path.join(OUTPUT_PATH, f'results_{year}.csv'), index=False)

### Unemployment data

In [16]:
unemployment = pd.read_excel(os.path.join(PATH, 'ERS_Unemployment.xlsx'), header=4)

In [17]:
rates_cols = ['Unemployment_rate_2000',
              'Unemployment_rate_2004',
              'Unemployment_rate_2008',
              'Unemployment_rate_2012',
              'Unemployment_rate_2016',
              'Unemployment_rate_2020']

In [18]:
area_cols = ['State', 'Area_name', 'FIPS_Code']

In [19]:
unemployment_ey = unemployment[area_cols + rates_cols]

In [20]:
unemployment_ey.sample(2)

,State,Area_name,FIPS_Code,Unemployment_rate_2000,Unemployment_rate_2004,Unemployment_rate_2008,Unemployment_rate_2012,Unemployment_rate_2016,Unemployment_rate_2020
1757,NE,"Red Willow County, NE",31145,2.6,3.5,2.8,3.2,2.8,3.3
58,AL,"Russell County, AL",1113,4.7,6.5,8.3,9.0,5.6,5.1


In [21]:
unemployment_ey_melted = pd.melt(unemployment_ey, 
                                 id_vars=area_cols, 
                                 value_vars=rates_cols, 
                                 value_name='unemployment',
                                 var_name='year')
unemployment_ey_melted['year'] = [x.split('_')[2] for x in unemployment_ey_melted['year']]
unemployment_ey_melted['year'] = unemployment_ey_melted['year'].astype(int)
unemployment_ey_melted.rename(columns={'FIPS_Code': 'county_fips'}, inplace=True)
unemployment_ey_melted = unemployment_ey_melted[['county_fips', 'year', 'unemployment']]

In [22]:
unemployment_ey_melted.sample()

,county_fips,year,unemployment
16870,13189,2020,7.8


Export unemployment data

In [23]:
for year in unemployment_ey_melted['year'].unique():
    
    df_temp = unemployment_ey_melted[unemployment_ey_melted['year'] == year]
    df_temp.to_csv(os.path.join(OUTPUT_PATH, f'unemployment_{year}.csv'), index=False)

#### Population

In [24]:
population = pd.read_excel(os.path.join(PATH, 'ERS_PopulationEstimates.xls'), header=2)

In [25]:
population.sample()

,FIPStxt,State,Area_Name,Rural-urban_Continuum Code_2003,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2003,Urban_Influence_Code_2013,Economic_typology_2015,CENSUS_2010_POP,ESTIMATES_BASE_2010,POP_ESTIMATE_2010,POP_ESTIMATE_2011,POP_ESTIMATE_2012,POP_ESTIMATE_2013,POP_ESTIMATE_2014,POP_ESTIMATE_2015,POP_ESTIMATE_2016,POP_ESTIMATE_2017,POP_ESTIMATE_2018,POP_ESTIMATE_2019,N_POP_CHG_2010,N_POP_CHG_2011,N_POP_CHG_2012,N_POP_CHG_2013,N_POP_CHG_2014,N_POP_CHG_2015,N_POP_CHG_2016,N_POP_CHG_2017,N_POP_CHG_2018,N_POP_CHG_2019,Births_2010,Births_2011,Births_2012,Births_2013,Births_2014,Births_2015,Births_2016,Births_2017,Births_2018,Births_2019,Deaths_2010,Deaths_2011,Deaths_2012,Deaths_2013,Deaths_2014,Deaths_2015,Deaths_2016,Deaths_2017,Deaths_2018,Deaths_2019,NATURAL_INC_2010,NATURAL_INC_2011,NATURAL_INC_2012,NATURAL_INC_2013,NATURAL_INC_2014,NATURAL_INC_2015,NATURAL_INC_2016,NATURAL_INC_2017,NATURAL_INC_2018,NATURAL_INC_2019,INTERNATIONAL_MIG_2010,INTERNATIONAL_MIG_2011,INTERNATIONAL_MIG_2012,INTERNATIONAL_MIG_2013,INTERNATIONAL_MIG_2014,INTERNATIONAL_MIG_2015,INTERNATIONAL_MIG_2016,INTERNATIONAL_MIG_2017,INTERNATIONAL_MIG_2018,INTERNATIONAL_MIG_2019,DOMESTIC_MIG_2010,DOMESTIC_MIG_2011,DOMESTIC_MIG_2012,DOMESTIC_MIG_2013,DOMESTIC_MIG_2014,DOMESTIC_MIG_2015,DOMESTIC_MIG_2016,DOMESTIC_MIG_2017,DOMESTIC_MIG_2018,DOMESTIC_MIG_2019,NET_MIG_2010,NET_MIG_2011,NET_MIG_2012,NET_MIG_2013,NET_MIG_2014,NET_MIG_2015,NET_MIG_2016,NET_MIG_2017,NET_MIG_2018,NET_MIG_2019,RESIDUAL_2010,RESIDUAL_2011,RESIDUAL_2012,RESIDUAL_2013,RESIDUAL_2014,RESIDUAL_2015,RESIDUAL_2016,RESIDUAL_2017,RESIDUAL_2018,RESIDUAL_2019,GQ_ESTIMATES_BASE_2010,GQ_ESTIMATES_2010,GQ_ESTIMATES_2011,GQ_ESTIMATES_2012,GQ_ESTIMATES_2013,GQ_ESTIMATES_2014,GQ_ESTIMATES_2015,GQ_ESTIMATES_2016,GQ_ESTIMATES_2017,GQ_ESTIMATES_2018,GQ_ESTIMATES_2019,R_birth_2011,R_birth_2012,R_birth_2013,R_birth_2014,R_birth_2015,R_birth_2016,R_birth_2017,R_birth_2018,R_birth_2019,R_death_2011,R_death_2012,R_death_2013,R_death_2014,R_death_2015,R_death_2016,R_death_2017,R_death_2018,R_death_2019,R_NATURAL_INC_2011,R_NATURAL_INC_2012,R_NATURAL_INC_2013,R_NATURAL_INC_2014,R_NATURAL_INC_2015,R_NATURAL_INC_2016,R_NATURAL_INC_2017,R_NATURAL_INC_2018,R_NATURAL_INC_2019,R_INTERNATIONAL_MIG_2011,R_INTERNATIONAL_MIG_2012,R_INTERNATIONAL_MIG_2013,R_INTERNATIONAL_MIG_2014,R_INTERNATIONAL_MIG_2015,R_INTERNATIONAL_MIG_2016,R_INTERNATIONAL_MIG_2017,R_INTERNATIONAL_MIG_2018,R_INTERNATIONAL_MIG_2019,R_DOMESTIC_MIG_2011,R_DOMESTIC_MIG_2012,R_DOMESTIC_MIG_2013,R_DOMESTIC_MIG_2014,R_DOMESTIC_MIG_2015,R_DOMESTIC_MIG_2016,R_DOMESTIC_MIG_2017,R_DOMESTIC_MIG_2018,R_DOMESTIC_MIG_2019,R_NET_MIG_2011,R_NET_MIG_2012,R_NET_MIG_2013,R_NET_MIG_2014,R_NET_MIG_2015,R_NET_MIG_2016,R_NET_MIG_2017,R_NET_MIG_2018,R_NET_MIG_2019
181,5131,AR,Sebastian County,2.0,2.0,2.0,2.0,3.0,125744,125740,125737,127010,127571,127173,126733,127385,127385,127786,127570,127827,-3.0,1273.0,561.0,-398.0,-440.0,652.0,0.0,401.0,-216.0,257.0,414.0,1763.0,1763.0,1626.0,1696.0,1677.0,1667.0,1791.0,1774.0,1780.0,311.0,1195.0,1232.0,1264.0,1265.0,1254.0,1292.0,1287.0,1347.0,1276.0,103.0,568.0,531.0,362.0,431.0,423.0,375.0,504.0,427.0,504.0,35.0,228.0,189.0,64.0,70.0,115.0,95.0,26.0,-50.0,-51.0,-133.0,478.0,-133.0,-827.0,-943.0,130.0,-467.0,-124.0,-590.0,-194.0,-98.0,706.0,56.0,-763.0,-873.0,245.0,-372.0,-98.0,-640.0,-245.0,-8.0,-1.0,-26.0,3.0,2.0,-16.0,-3.0,-5.0,-3.0,-2.0,2235.0,2235.0,2310.0,2414.0,2407.0,2397.0,2380.0,2302.0,2275.0,2242.0,2242.0,13.95071,13.850209,12.765757,13.359275,13.198593,13.086313,14.037645,13.894328,13.939083,9.456096,9.678648,9.923688,9.964318,9.869431,10.142481,10.087353,10.549977,9.992287,4.494613,4.17156,2.842069,3.394957,3.329162,2.943832,3.950292,3.344351,3.946797,1.804176,1.484793,0.502465,0.551385,0.905091,0.745771,0.203785,-0.39161,-0.399378,3.782439,-1.044854,-6.492793,-7.427946,1.023147,-3.666052,-0.971897,-4.621,-1.519203,5.586614,0.439939,-5.990328,-6.876561,1.928238,-2.920281,-0.768112,-5.01261,-1.918582


We obtained the rates for natural increase in population, international migration, domestic migration, and net migration for the years 2012, 2016, and we substituted the value of 2019 for 2020. 

In [26]:
years_population = ['2012', '2016', '2019']

In [27]:
rate_nat_inc_cols = ['R_NATURAL_INC_' + str(x) for x in years_population]
rate_int_mig_cols = ['R_INTERNATIONAL_MIG_' + str(x) for x in years_population]
rate_dom_mig_cols = ['R_DOMESTIC_MIG_' + str(x) for x in years_population]
rate_net_mig_cols = ['R_NET_MIG_' + str(x) for x in years_population]

In [28]:
population_dict = {'rate_natural_increase_population': rate_nat_inc_cols,
                   'rate_international_migration': rate_int_mig_cols,
                   'rate_domestic_migration': rate_dom_mig_cols,
                   'rate_net_migration': rate_net_mig_cols}

In [29]:
for key in population_dict.keys():
    
    cols = population_dict[key]
    temp2 = pd.melt(population[['FIPStxt'] + cols], 
                    id_vars=['FIPStxt'], 
                    value_vars=cols, 
                    value_name=key,
                    var_name='year')
    temp2['year'] = [x.split("_")[3] for x in temp2['year']]
    temp2['year'] = ['2020' if x == '2019' else x for x in temp2['year']]
    
    for year in temp2['year'].unique():
        df_temp = temp2[temp2['year'] == year]
        df_temp.to_csv(os.path.join(OUTPUT_PATH, f'{key}_{year}.csv'), index=False)
        

In [30]:
natural_inc_year = pd.melt(population[['FIPStxt'] + rate_nat_inc_cols], 
                           id_vars=['FIPStxt'], 
                           value_vars=rate_nat_inc_cols, 
                           value_name='natural_increase_in_pop',
                           var_name='year')

natural_inc_year['year'] = [x.split("_")[3] for x in natural_inc_year['year']]
natural_inc_year['year'] = ['2020' if x == '2019' else x for x in natural_inc_year['year']]